In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 

In [6]:
def margin_req(strike,price,premium,n_options = 100):
    # let's find the maximum between 3 choices:
    max_choice = np.max([0.3 * price - np.max([price - strike,0]),
           0.1 * strike,
           250 * n_options])
    return (max_choice + premium) * 100


In [7]:
margin_req(585,579.78,7.75,n_options=27)

675775.0

In [ ]:
np.max